## Process CSV file into a CSV file that can be turned into an ASlib scenario: HH Features

In [1]:
import pandas as pd
df = pd.read_csv('C:/Users/Jasmin/Documents/MSc-Thesis/Feature_Extraction/feature_data_rp/RP_data_07.csv')

In [2]:
df.head()

,n_vars,n_constr,n_nzcnt,nq_vars,nq_constr,nq_nzcnt,lp_avg,lp_l2_avg,lp_linf,lp_objval,...,impliedBoundCuts,flowCuts,mixedIntegerRoundingCuts,gomoryFractionalCuts,time_relax,time_VCG0,time_VCG1,time_VCG2,cplex_prob_time,INSTANCE_NAME
0,3005.0,4440.0,532558.0,0.0,0.0,0.0,0.110512,0.191534,0.497890,0.0,...,0.0,0.0,9.0,4.0,2.05000,0.000000,0.200000,0.183333,24.3167,mip_1.txt
1,2971.0,4372.0,505477.0,0.0,0.0,0.0,0.134393,0.221089,0.499550,0.0,...,0.0,0.0,20.0,10.0,2.16667,0.000000,0.216667,0.183333,25.0000,mip_10.txt
2,2967.0,4364.0,502291.0,0.0,0.0,0.0,0.106764,0.190398,0.498555,0.0,...,0.0,0.0,21.0,10.0,2.00000,0.000000,0.183333,0.183333,22.7500,mip_100.txt
3,3011.0,4452.0,537337.0,0.0,0.0,0.0,0.230292,0.322005,0.499855,0.0,...,0.0,0.0,14.0,7.0,3.36667,0.016667,0.200000,0.200000,23.4500,mip_1000.txt
4,2897.0,4224.0,446536.0,0.0,0.0,0.0,0.133350,0.217471,0.498412,0.0,...,0.0,0.0,95.0,47.0,2.06667,0.016667,0.183333,0.166667,23.5333,mip_10000.txt


In [3]:
df['INSTANCE_NAME'] = 'mip' + df['INSTANCE_NAME'].astype('str').astype('str').str.extractall('(\d+)').unstack().fillna('').sum(axis=1).astype('int').astype('str')
df.head()

,n_vars,n_constr,n_nzcnt,nq_vars,nq_constr,nq_nzcnt,lp_avg,lp_l2_avg,lp_linf,lp_objval,...,impliedBoundCuts,flowCuts,mixedIntegerRoundingCuts,gomoryFractionalCuts,time_relax,time_VCG0,time_VCG1,time_VCG2,cplex_prob_time,INSTANCE_NAME
0,3005.0,4440.0,532558.0,0.0,0.0,0.0,0.110512,0.191534,0.497890,0.0,...,0.0,0.0,9.0,4.0,2.05000,0.000000,0.200000,0.183333,24.3167,mip1
1,2971.0,4372.0,505477.0,0.0,0.0,0.0,0.134393,0.221089,0.499550,0.0,...,0.0,0.0,20.0,10.0,2.16667,0.000000,0.216667,0.183333,25.0000,mip10
2,2967.0,4364.0,502291.0,0.0,0.0,0.0,0.106764,0.190398,0.498555,0.0,...,0.0,0.0,21.0,10.0,2.00000,0.000000,0.183333,0.183333,22.7500,mip100
3,3011.0,4452.0,537337.0,0.0,0.0,0.0,0.230292,0.322005,0.499855,0.0,...,0.0,0.0,14.0,7.0,3.36667,0.016667,0.200000,0.200000,23.4500,mip1000
4,2897.0,4224.0,446536.0,0.0,0.0,0.0,0.133350,0.217471,0.498412,0.0,...,0.0,0.0,95.0,47.0,2.06667,0.016667,0.183333,0.166667,23.5333,mip10000


### Undersample the data such that there is an equal number of instances where each of the configurations is best

In [13]:
runtimes =  pd.read_excel('C:/Users/Jasmin/Documents/MSc-Thesis/Data/Runtime/Results-MIPVerify_SDPdMLPa.xlsx')
runtimes = runtimes[['SampleNumber', 'Config_best']]
runtimes.rename(columns={'SampleNumber': 'INSTANCE_NAME'}, inplace=True)
runtimes['INSTANCE_NAME'] = 'mip' + runtimes.INSTANCE_NAME.astype('str')
runtimes

,INSTANCE_NAME,Config_best
0,mip1,SolveTime_conf2
1,mip2,TIMEOUT
2,mip3,SolveTime_conf1
3,mip4,SolveTime_conf3
4,mip5,SolveTime_conf1
...,...,...
9995,mip9996,SolveTime_default
9996,mip9997,SolveTime_default
9997,mip9998,SolveTime_default
9998,mip9999,SolveTime_conf1


In [16]:
df = pd.merge(df, runtimes, on='INSTANCE_NAME')

In [18]:
print(df['Config_best'].value_counts())

SolveTime_conf3      3281
SolveTime_conf2      2303
SolveTime_default    1829
TIMEOUT              1498
SolveTime_conf1      1089
Name: Config_best, dtype: int64


In [19]:
conf3, conf2, confdefault, timeouts, conf1 = df['Config_best'].value_counts()

c4 = df[df['Config_best'] == 'TIMEOUT']
c3 = df[df['Config_best'] == 'SolveTime_conf3']
c2 = df[df['Config_best'] == 'SolveTime_conf2']
c1 = df[df['Config_best'] == 'SolveTime_conf1']
c0 = df[df['Config_best'] == 'SolveTime_default']
df4 = c4.sample(timeouts)
df3 = c3.sample(conf3)
df2 = c2.sample(conf2)
df1 = c1.sample(conf1)
df0 = c0.sample(confdefault)
#print(df0.shape)
#print(df1.shape)
#print(df2.shape)
sampled4 = df4.sample(n=len(df1), random_state=1)
sampled3 = df3.sample(n=len(df1), random_state=1)
sampled2 = df2.sample(n=len(df1), random_state=1)
#print(sampled1.shape)
sampled0 = df0.sample(n=len(df1), random_state=1)
#print(sampled0.shape)
undersampled_df = pd.concat([df1, sampled0, sampled2, sampled3, sampled4])
print(undersampled_df.shape)

(5445, 149)


In [21]:
df = undersampled_df
print(df['Config_best'].value_counts())

SolveTime_conf1      1089
SolveTime_default    1089
SolveTime_conf2      1089
SolveTime_conf3      1089
TIMEOUT              1089
Name: Config_best, dtype: int64


In [45]:
df

,n_vars,n_constr,n_nzcnt,nq_vars,nq_constr,nq_nzcnt,lp_avg,lp_l2_avg,lp_linf,lp_objval,...,flowCuts,mixedIntegerRoundingCuts,gomoryFractionalCuts,time_relax,time_VCG0,time_VCG1,time_VCG2,cplex_prob_time,INSTANCE_NAME,Config_best
2004,2935.0,4300.0,476803.0,0.0,0.0,0.0,0.132970,0.222440,0.496111,0.0,...,0.0,92.0,47.0,1.83333,0.000000,0.183333,0.166667,22.3667,mip2801,SolveTime_conf1
3583,2947.0,4324.0,486361.0,0.0,0.0,0.0,0.127408,0.213098,0.498799,0.0,...,0.0,8.0,4.0,1.28333,0.016667,0.183333,0.183333,23.0000,mip4222,SolveTime_conf1
1720,2973.0,4376.0,507070.0,0.0,0.0,0.0,0.123476,0.208722,0.497115,0.0,...,0.0,18.0,9.0,2.11667,0.000000,0.200000,0.200000,26.5000,mip2546,SolveTime_conf1
6674,2945.0,4320.0,484768.0,0.0,0.0,0.0,0.125174,0.210313,0.498951,0.0,...,0.0,52.0,26.0,1.78333,0.000000,0.166667,0.166667,22.8500,mip7004,SolveTime_conf1
8834,2817.0,4064.0,382816.0,0.0,0.0,0.0,0.136906,0.226524,0.495137,0.0,...,0.0,14.0,6.0,1.65000,0.000000,0.150000,0.150000,26.7333,mip895,SolveTime_conf1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5618,2963.0,4356.0,499105.0,0.0,0.0,0.0,0.120475,0.206968,0.498779,0.0,...,0.0,36.0,18.0,2.11667,0.000000,0.200000,0.183333,22.4333,mip6054,TIMEOUT
1459,3035.0,4500.0,556453.0,0.0,0.0,0.0,0.145017,0.225093,0.497666,0.0,...,0.0,64.0,32.0,2.25000,0.016667,0.200000,0.216667,22.6167,mip2310,TIMEOUT
1709,2941.0,4312.0,481582.0,0.0,0.0,0.0,0.219942,0.316728,0.499774,0.0,...,0.0,42.0,22.0,3.30000,0.000000,0.183333,0.183333,25.0667,mip2536,TIMEOUT
1987,3061.0,4552.0,577162.0,0.0,0.0,0.0,0.213396,0.309034,0.499601,0.0,...,0.0,44.0,15.0,3.76667,0.000000,0.200000,0.200000,24.0500,mip2787,TIMEOUT


### Add rows as comma seperated data to feature_values.arff file

In [22]:
# first make sure all column names are correct and isolate timing features
timing_df = df[['time_relax', 'time_VCG0', 'time_VCG1','time_VCG2','cplex_prob_time']]
timing_df

,time_relax,time_VCG0,time_VCG1,time_VCG2,cplex_prob_time
2004,1.83333,0.000000,0.183333,0.166667,22.3667
3583,1.28333,0.016667,0.183333,0.183333,23.0000
1720,2.11667,0.000000,0.200000,0.200000,26.5000
6674,1.78333,0.000000,0.166667,0.166667,22.8500
8834,1.65000,0.000000,0.150000,0.150000,26.7333
...,...,...,...,...,...
5618,2.11667,0.000000,0.200000,0.183333,22.4333
1459,2.25000,0.016667,0.200000,0.216667,22.6167
1709,3.30000,0.000000,0.183333,0.183333,25.0667
1987,3.76667,0.000000,0.200000,0.200000,24.0500


In [23]:
# first make sure all column names are correct and isolate timing features
features_df = df.drop(columns=timing_df.columns)
features_df

,n_vars,n_constr,n_nzcnt,nq_vars,nq_constr,nq_nzcnt,lp_avg,lp_l2_avg,lp_linf,lp_objval,...,new_col,new_nonzero,clique_table,cliqueCuts,impliedBoundCuts,flowCuts,mixedIntegerRoundingCuts,gomoryFractionalCuts,INSTANCE_NAME,Config_best
2004,2935.0,4300.0,476803.0,0.0,0.0,0.0,0.132970,0.222440,0.496111,0.0,...,1547.0,472533.0,90.0,0.0,3.0,0.0,92.0,47.0,mip2801,SolveTime_conf1
3583,2947.0,4324.0,486361.0,0.0,0.0,0.0,0.127408,0.213098,0.498799,0.0,...,1506.0,481873.0,26.0,0.0,0.0,0.0,8.0,4.0,mip4222,SolveTime_conf1
1720,2973.0,4376.0,507070.0,0.0,0.0,0.0,0.123476,0.208722,0.497115,0.0,...,1559.0,502677.0,146.0,0.0,0.0,0.0,18.0,9.0,mip2546,SolveTime_conf1
6674,2945.0,4320.0,484768.0,0.0,0.0,0.0,0.125174,0.210313,0.498951,0.0,...,1533.0,480397.0,262.0,0.0,0.0,0.0,52.0,26.0,mip7004,SolveTime_conf1
8834,2817.0,4064.0,382816.0,0.0,0.0,0.0,0.136906,0.226524,0.495137,0.0,...,1304.0,378105.0,145.0,0.0,0.0,0.0,14.0,6.0,mip895,SolveTime_conf1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5618,2963.0,4356.0,499105.0,0.0,0.0,0.0,0.120475,0.206968,0.498779,0.0,...,1607.0,494949.0,20.0,0.0,0.0,0.0,36.0,18.0,mip6054,TIMEOUT
1459,3035.0,4500.0,556453.0,0.0,0.0,0.0,0.145017,0.225093,0.497666,0.0,...,1600.0,551945.0,29.0,0.0,0.0,0.0,64.0,32.0,mip2310,TIMEOUT
1709,2941.0,4312.0,481582.0,0.0,0.0,0.0,0.219942,0.316728,0.499774,0.0,...,1572.0,477385.0,425.0,0.0,0.0,0.0,42.0,22.0,mip2536,TIMEOUT
1987,3061.0,4552.0,577162.0,0.0,0.0,0.0,0.213396,0.309034,0.499601,0.0,...,1550.0,572337.0,225.0,0.0,215.0,0.0,44.0,15.0,mip2787,TIMEOUT


In [24]:
# first make sure all column names are correct --> instance id and repetitions
features_df.rename(columns={"INSTANCE_NAME":"instance_id"}, inplace=True)

## Clean features for better predictions

In [25]:
nunique = features_df.nunique()
cols_to_drop = nunique[nunique == 1].index
features_df = features_df.drop(cols_to_drop, axis=1)
features_df = features_df.T.drop_duplicates().T
features_df

,n_vars,n_constr,n_nzcnt,lp_avg,lp_l2_avg,lp_linf,num_i_variables,num_c_variables,ratio_i_variables,ratio_c_variables,...,new_col,new_nonzero,clique_table,cliqueCuts,impliedBoundCuts,flowCuts,mixedIntegerRoundingCuts,gomoryFractionalCuts,instance_id,Config_best
2004,2935.0,4300.0,476803.0,0.13297,0.22244,0.496111,295.0,2640.0,0.100511,0.899489,...,1547.0,472533.0,90.0,0.0,3.0,0.0,92.0,47.0,mip2801,SolveTime_conf1
3583,2947.0,4324.0,486361.0,0.127408,0.213098,0.498799,301.0,2646.0,0.102138,0.897862,...,1506.0,481873.0,26.0,0.0,0.0,0.0,8.0,4.0,mip4222,SolveTime_conf1
1720,2973.0,4376.0,507070.0,0.123476,0.208722,0.497115,314.0,2659.0,0.105617,0.894383,...,1559.0,502677.0,146.0,0.0,0.0,0.0,18.0,9.0,mip2546,SolveTime_conf1
6674,2945.0,4320.0,484768.0,0.125174,0.210313,0.498951,300.0,2645.0,0.101868,0.898132,...,1533.0,480397.0,262.0,0.0,0.0,0.0,52.0,26.0,mip7004,SolveTime_conf1
8834,2817.0,4064.0,382816.0,0.136906,0.226524,0.495137,236.0,2581.0,0.083777,0.916223,...,1304.0,378105.0,145.0,0.0,0.0,0.0,14.0,6.0,mip895,SolveTime_conf1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5618,2963.0,4356.0,499105.0,0.120475,0.206968,0.498779,309.0,2654.0,0.104286,0.895714,...,1607.0,494949.0,20.0,0.0,0.0,0.0,36.0,18.0,mip6054,TIMEOUT
1459,3035.0,4500.0,556453.0,0.145017,0.225093,0.497666,345.0,2690.0,0.113674,0.886326,...,1600.0,551945.0,29.0,0.0,0.0,0.0,64.0,32.0,mip2310,TIMEOUT
1709,2941.0,4312.0,481582.0,0.219942,0.316728,0.499774,298.0,2643.0,0.101326,0.898674,...,1572.0,477385.0,425.0,0.0,0.0,0.0,42.0,22.0,mip2536,TIMEOUT
1987,3061.0,4552.0,577162.0,0.213396,0.309034,0.499601,358.0,2703.0,0.116955,0.883045,...,1550.0,572337.0,225.0,0.0,215.0,0.0,44.0,15.0,mip2787,TIMEOUT


In [27]:
features_df = features_df.drop('Config_best', axis=1)

In [28]:
features_df["repetition"] = 1
features_df

,n_vars,n_constr,n_nzcnt,lp_avg,lp_l2_avg,lp_linf,num_i_variables,num_c_variables,ratio_i_variables,ratio_c_variables,...,new_col,new_nonzero,clique_table,cliqueCuts,impliedBoundCuts,flowCuts,mixedIntegerRoundingCuts,gomoryFractionalCuts,instance_id,repetition
2004,2935.0,4300.0,476803.0,0.13297,0.22244,0.496111,295.0,2640.0,0.100511,0.899489,...,1547.0,472533.0,90.0,0.0,3.0,0.0,92.0,47.0,mip2801,1
3583,2947.0,4324.0,486361.0,0.127408,0.213098,0.498799,301.0,2646.0,0.102138,0.897862,...,1506.0,481873.0,26.0,0.0,0.0,0.0,8.0,4.0,mip4222,1
1720,2973.0,4376.0,507070.0,0.123476,0.208722,0.497115,314.0,2659.0,0.105617,0.894383,...,1559.0,502677.0,146.0,0.0,0.0,0.0,18.0,9.0,mip2546,1
6674,2945.0,4320.0,484768.0,0.125174,0.210313,0.498951,300.0,2645.0,0.101868,0.898132,...,1533.0,480397.0,262.0,0.0,0.0,0.0,52.0,26.0,mip7004,1
8834,2817.0,4064.0,382816.0,0.136906,0.226524,0.495137,236.0,2581.0,0.083777,0.916223,...,1304.0,378105.0,145.0,0.0,0.0,0.0,14.0,6.0,mip895,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5618,2963.0,4356.0,499105.0,0.120475,0.206968,0.498779,309.0,2654.0,0.104286,0.895714,...,1607.0,494949.0,20.0,0.0,0.0,0.0,36.0,18.0,mip6054,1
1459,3035.0,4500.0,556453.0,0.145017,0.225093,0.497666,345.0,2690.0,0.113674,0.886326,...,1600.0,551945.0,29.0,0.0,0.0,0.0,64.0,32.0,mip2310,1
1709,2941.0,4312.0,481582.0,0.219942,0.316728,0.499774,298.0,2643.0,0.101326,0.898674,...,1572.0,477385.0,425.0,0.0,0.0,0.0,42.0,22.0,mip2536,1
1987,3061.0,4552.0,577162.0,0.213396,0.309034,0.499601,358.0,2703.0,0.116955,0.883045,...,1550.0,572337.0,225.0,0.0,215.0,0.0,44.0,15.0,mip2787,1


In [29]:
# Change position of columns --> instance_id and repetition go first
cols = features_df.columns.tolist()
cols

['n_vars',
 'n_constr',
 'n_nzcnt',
 'lp_avg',
 'lp_l2_avg',
 'lp_linf',
 'num_i_variables',
 'num_c_variables',
 'ratio_i_variables',
 'ratio_c_variables',
 'rhs_c_0_avg',
 'rhs_c_0_varcoef',
 'rhs_c_1_avg',
 'rhs_c_1_varcoef',
 'rhs_c_2_avg',
 'rhs_c_2_varcoef',
 'vcg_constr_deg0_avg',
 'vcg_constr_deg0_varcoef',
 'vcg_constr_weight0_avg',
 'vcg_constr_weight0_varcoef',
 'vcg_var_weight0_avg',
 'vcg_var_weight0_varcoef',
 'A_ij_normalized0_avg',
 'A_ij_normalized0_varcoef',
 'a_normalized_varcoefs0_avg',
 'a_normalized_varcoefs0_varcoef',
 'vcg_constr_deg1_avg',
 'vcg_constr_deg1_varcoef',
 'vcg_var_deg1_avg',
 'vcg_var_deg1_varcoef',
 'vcg_var_deg1_q90mq10',
 'vcg_constr_weight1_avg',
 'vcg_constr_weight1_varcoef',
 'vcg_var_weight1_avg',
 'vcg_var_weight1_varcoef',
 'A_ij_normalized1_avg',
 'A_ij_normalized1_varcoef',
 'a_normalized_varcoefs1_avg',
 'a_normalized_varcoefs1_varcoef',
 'obj_coefs1_avg',
 'obj_coefs1_std',
 'obj_coef_per_constr1_avg',
 'obj_coef_per_constr1_std',
 'ob

In [30]:
# Change position of columns --> instance_id and repetition go first
cols = cols[-2:] + cols[:-2]
cols

['instance_id',
 'repetition',
 'n_vars',
 'n_constr',
 'n_nzcnt',
 'lp_avg',
 'lp_l2_avg',
 'lp_linf',
 'num_i_variables',
 'num_c_variables',
 'ratio_i_variables',
 'ratio_c_variables',
 'rhs_c_0_avg',
 'rhs_c_0_varcoef',
 'rhs_c_1_avg',
 'rhs_c_1_varcoef',
 'rhs_c_2_avg',
 'rhs_c_2_varcoef',
 'vcg_constr_deg0_avg',
 'vcg_constr_deg0_varcoef',
 'vcg_constr_weight0_avg',
 'vcg_constr_weight0_varcoef',
 'vcg_var_weight0_avg',
 'vcg_var_weight0_varcoef',
 'A_ij_normalized0_avg',
 'A_ij_normalized0_varcoef',
 'a_normalized_varcoefs0_avg',
 'a_normalized_varcoefs0_varcoef',
 'vcg_constr_deg1_avg',
 'vcg_constr_deg1_varcoef',
 'vcg_var_deg1_avg',
 'vcg_var_deg1_varcoef',
 'vcg_var_deg1_q90mq10',
 'vcg_constr_weight1_avg',
 'vcg_constr_weight1_varcoef',
 'vcg_var_weight1_avg',
 'vcg_var_weight1_varcoef',
 'A_ij_normalized1_avg',
 'A_ij_normalized1_varcoef',
 'a_normalized_varcoefs1_avg',
 'a_normalized_varcoefs1_varcoef',
 'obj_coefs1_avg',
 'obj_coefs1_std',
 'obj_coef_per_constr1_avg',
 '

In [31]:
features_df = features_df[cols]
features_df

,instance_id,repetition,n_vars,n_constr,n_nzcnt,lp_avg,lp_l2_avg,lp_linf,num_i_variables,num_c_variables,...,pre_t,new_row,new_col,new_nonzero,clique_table,cliqueCuts,impliedBoundCuts,flowCuts,mixedIntegerRoundingCuts,gomoryFractionalCuts
2004,mip2801,1,2935.0,4300.0,476803.0,0.13297,0.22244,0.496111,295.0,2640.0,...,0.25,2022.0,1547.0,472533.0,90.0,0.0,3.0,0.0,92.0,47.0
3583,mip4222,1,2947.0,4324.0,486361.0,0.127408,0.213098,0.498799,301.0,2646.0,...,0.25,1934.0,1506.0,481873.0,26.0,0.0,0.0,0.0,8.0,4.0
1720,mip2546,1,2973.0,4376.0,507070.0,0.123476,0.208722,0.497115,314.0,2659.0,...,0.26,2027.0,1559.0,502677.0,146.0,0.0,0.0,0.0,18.0,9.0
6674,mip7004,1,2945.0,4320.0,484768.0,0.125174,0.210313,0.498951,300.0,2645.0,...,0.25,1989.0,1533.0,480397.0,262.0,0.0,0.0,0.0,52.0,26.0
8834,mip895,1,2817.0,4064.0,382816.0,0.136906,0.226524,0.495137,236.0,2581.0,...,0.2,1595.0,1304.0,378105.0,145.0,0.0,0.0,0.0,14.0,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5618,mip6054,1,2963.0,4356.0,499105.0,0.120475,0.206968,0.498779,309.0,2654.0,...,0.26,2128.0,1607.0,494949.0,20.0,0.0,0.0,0.0,36.0,18.0
1459,mip2310,1,3035.0,4500.0,556453.0,0.145017,0.225093,0.497666,345.0,2690.0,...,0.29,2078.0,1600.0,551945.0,29.0,0.0,0.0,0.0,64.0,32.0
1709,mip2536,1,2941.0,4312.0,481582.0,0.219942,0.316728,0.499774,298.0,2643.0,...,0.25,2069.0,1572.0,477385.0,425.0,0.0,0.0,0.0,42.0,22.0
1987,mip2787,1,3061.0,4552.0,577162.0,0.213396,0.309034,0.499601,358.0,2703.0,...,0.29,1965.0,1550.0,572337.0,225.0,0.0,215.0,0.0,44.0,15.0


In [32]:
id_order = list(features_df.instance_id)

In [33]:
test_data = {'test1':[1,2,3],'test2':[4,5,6],'test3':['a','b','c']}
example_test_df = pd.DataFrame(test_data)
example_test_df

,test1,test2,test3
0,1,4,a
1,2,5,b
2,3,6,c


In [34]:
for index,row in example_test_df.iterrows():
    print(str(list(row)).strip("[]").replace("'", ""))

1, 4, a
2, 5, b
3, 6, c


In [35]:
# @ATTRIBUTE instance_id STRING
# Add all attributes in correct order

with open("MIPVerifySDP_cleanARPsamplefeatureset/feature_values.arff","a") as file:
    for col in features_df.columns:
        file.write("\n")
        if col == 'instance_id':
            newrow = '@ATTRIBUTE ' + col + ' STRING'
        else:
            newrow = '@ATTRIBUTE ' + col + ' NUMERIC'
        file.write(newrow)

    file.close()

In [36]:
# Get comma seperated rows and iteratively add to feature_values.arff
with open("MIPVerifySDP_cleanARPsamplefeatureset/feature_values.arff","a") as file:
    for index,row in features_df.iterrows():
        file.write("\n")
        newrow = str(list(row)).strip("[]").replace("'", "")
        file.write(newrow)

    file.close()

In [37]:
# double check the column order
for i in features_df.columns:
    print(i)

instance_id
repetition
n_vars
n_constr
n_nzcnt
lp_avg
lp_l2_avg
lp_linf
num_i_variables
num_c_variables
ratio_i_variables
ratio_c_variables
rhs_c_0_avg
rhs_c_0_varcoef
rhs_c_1_avg
rhs_c_1_varcoef
rhs_c_2_avg
rhs_c_2_varcoef
vcg_constr_deg0_avg
vcg_constr_deg0_varcoef
vcg_constr_weight0_avg
vcg_constr_weight0_varcoef
vcg_var_weight0_avg
vcg_var_weight0_varcoef
A_ij_normalized0_avg
A_ij_normalized0_varcoef
a_normalized_varcoefs0_avg
a_normalized_varcoefs0_varcoef
vcg_constr_deg1_avg
vcg_constr_deg1_varcoef
vcg_var_deg1_avg
vcg_var_deg1_varcoef
vcg_var_deg1_q90mq10
vcg_constr_weight1_avg
vcg_constr_weight1_varcoef
vcg_var_weight1_avg
vcg_var_weight1_varcoef
A_ij_normalized1_avg
A_ij_normalized1_varcoef
a_normalized_varcoefs1_avg
a_normalized_varcoefs1_varcoef
obj_coefs1_avg
obj_coefs1_std
obj_coef_per_constr1_avg
obj_coef_per_constr1_std
obj_coef_per_sqr_constr1_avg
obj_coef_per_sqr_constr1_std
vcg_constr_deg2_avg
vcg_constr_deg2_varcoef
vcg_var_deg2_avg
vcg_var_deg2_varcoef
vcg_var_deg2_

## make description.txt

In [29]:
for i in features_df.columns:
    print('- ', i)

-  instance_id
-  repetition
-  n_vars
-  n_constr
-  n_nzcnt
-  lp_avg
-  lp_l2_avg
-  lp_linf
-  num_i_variables
-  num_c_variables
-  ratio_i_variables
-  ratio_c_variables
-  rhs_c_0_avg
-  rhs_c_0_varcoef
-  rhs_c_1_avg
-  rhs_c_1_varcoef
-  rhs_c_2_avg
-  rhs_c_2_varcoef
-  vcg_constr_deg0_avg
-  vcg_constr_deg0_varcoef
-  vcg_constr_weight0_avg
-  vcg_constr_weight0_varcoef
-  vcg_var_weight0_avg
-  vcg_var_weight0_varcoef
-  A_ij_normalized0_avg
-  A_ij_normalized0_varcoef
-  a_normalized_varcoefs0_avg
-  a_normalized_varcoefs0_varcoef
-  vcg_constr_deg1_avg
-  vcg_constr_deg1_varcoef
-  vcg_var_deg1_avg
-  vcg_var_deg1_varcoef
-  vcg_var_deg1_q90mq10
-  vcg_constr_weight1_avg
-  vcg_constr_weight1_varcoef
-  vcg_var_weight1_avg
-  vcg_var_weight1_varcoef
-  A_ij_normalized1_avg
-  A_ij_normalized1_varcoef
-  a_normalized_varcoefs1_avg
-  a_normalized_varcoefs1_varcoef
-  obj_coefs1_avg
-  obj_coefs1_std
-  obj_coef_per_constr1_avg
-  obj_coef_per_constr1_std
-  obj_coef_per_sq

### Make the feature_runstatus.arff file

In [38]:
runstatus_df = pd.DataFrame({'instance_id': id_order, 'repetition':[1]*5445, 'trivial':['ok']*5445, 
                             'lpbased':['ok']*5445,'vcg0':['ok']*5445,'vcg1':['ok']*5445,
                             'vcg2':['ok']*5445, 'probing':['ok']*5445})

runstatus_df

,instance_id,repetition,trivial,lpbased,vcg0,vcg1,vcg2,probing
0,mip2801,1,ok,ok,ok,ok,ok,ok
1,mip4222,1,ok,ok,ok,ok,ok,ok
2,mip2546,1,ok,ok,ok,ok,ok,ok
3,mip7004,1,ok,ok,ok,ok,ok,ok
4,mip895,1,ok,ok,ok,ok,ok,ok
...,...,...,...,...,...,...,...,...
5440,mip6054,1,ok,ok,ok,ok,ok,ok
5441,mip2310,1,ok,ok,ok,ok,ok,ok
5442,mip2536,1,ok,ok,ok,ok,ok,ok
5443,mip2787,1,ok,ok,ok,ok,ok,ok


In [39]:
with open("MIPVerifySDP_cleanARPsamplefeatureset/feature_runstatus.arff","a") as file:
    for index,row in runstatus_df.iterrows():
        file.write("\n")
        newrow = str(list(row)).strip("[]").replace("'", "")
        file.write(newrow)

    file.close()

### Now make feature_costs.arff file

In [50]:
costs_df = pd.DataFrame({'instance_id': id_order, 'repetition':[1]*5445})
costs_df['time_trivial'] = 0
costs_df

,instance_id,repetition,time_trivial
0,mip2801,1,0
1,mip4222,1,0
2,mip2546,1,0
3,mip7004,1,0
4,mip895,1,0
...,...,...,...
5440,mip6054,1,0
5441,mip2310,1,0
5442,mip2536,1,0
5443,mip2787,1,0


In [49]:
timing_df.reset_index(inplace=True, drop=True)

In [54]:
costs_df = pd.concat([costs_df,timing_df],axis=1)

In [55]:
costs_df

,instance_id,repetition,time_trivial,time_relax,time_VCG0,time_VCG1,time_VCG2,cplex_prob_time
0,mip2801,1,0,1.83333,0.000000,0.183333,0.166667,22.3667
1,mip4222,1,0,1.28333,0.016667,0.183333,0.183333,23.0000
2,mip2546,1,0,2.11667,0.000000,0.200000,0.200000,26.5000
3,mip7004,1,0,1.78333,0.000000,0.166667,0.166667,22.8500
4,mip895,1,0,1.65000,0.000000,0.150000,0.150000,26.7333
...,...,...,...,...,...,...,...,...
5440,mip6054,1,0,2.11667,0.000000,0.200000,0.183333,22.4333
5441,mip2310,1,0,2.25000,0.016667,0.200000,0.216667,22.6167
5442,mip2536,1,0,3.30000,0.000000,0.183333,0.183333,25.0667
5443,mip2787,1,0,3.76667,0.000000,0.200000,0.200000,24.0500


In [56]:
with open("MIPVerifySDP_cleanARPsamplefeatureset/feature_costs.arff","a") as file:
    for index,row in costs_df.iterrows():
        file.write("\n")
        newrow = str(list(row)).strip("[]").replace("'", "")
        file.write(newrow)

    file.close()

### Next step is creating algorithm_runs.arff

In [57]:
runtimes = pd.read_excel('C:/Users/Jasmin/Documents/MSc-Thesis/Data/Runtime/Results-MIPVerify_SDPdMLPa.xlsx')
runtimes

,SampleNumber,SolveTime_default,SolveStatus_default,SolveTime_conf1,SolveStatus_conf1,SolveTime_conf2,SolveStatus_conf2,SolveTime_conf3,SolveStatus_conf3,SolveTime_best,Config_best
0,1,104.393417,InfeasibleOrUnbounded,152.683291,InfeasibleOrUnbounded,44.068493,Infeasible,78.731996,Infeasible,44.068493,SolveTime_conf2
1,2,9600.323414,UserLimit,9600.080699,UserLimit,9600.184597,UserLimit,9600.010288,UserLimit,9600.010288,TIMEOUT
2,3,9600.157577,UserLimit,1258.149140,InfeasibleOrUnbounded,3025.400426,Infeasible,2938.074613,Infeasible,1258.149140,SolveTime_conf1
3,4,83.344657,InfeasibleOrUnbounded,170.613330,InfeasibleOrUnbounded,54.733004,Infeasible,41.134581,Infeasible,41.134581,SolveTime_conf3
4,5,5567.992290,InfeasibleOrUnbounded,220.795008,InfeasibleOrUnbounded,538.053177,Infeasible,300.046997,Infeasible,220.795008,SolveTime_conf1
...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,12.674720,InfeasibleOrUnbounded,18608.097787,UserLimit,25.863838,Infeasible,172.116870,Infeasible,12.674720,SolveTime_default
9996,9997,18.379950,InfeasibleOrUnbounded,110.131027,InfeasibleOrUnbounded,50.209194,Infeasible,110.876559,Infeasible,18.379950,SolveTime_default
9997,9998,75.684641,InfeasibleOrUnbounded,105.670172,InfeasibleOrUnbounded,106.346014,Infeasible,102.811690,Infeasible,75.684641,SolveTime_default
9998,9999,1669.967257,InfeasibleOrUnbounded,826.113838,InfeasibleOrUnbounded,1401.266455,Infeasible,1643.462584,Infeasible,826.113838,SolveTime_conf1


In [58]:
runtimes.rename(columns={'SampleNumber':'instance_id'},inplace=True)

In [59]:
runtimes['instance_id'] = 'mip' + runtimes['instance_id'].astype('str')
runtimes.head()

,instance_id,SolveTime_default,SolveStatus_default,SolveTime_conf1,SolveStatus_conf1,SolveTime_conf2,SolveStatus_conf2,SolveTime_conf3,SolveStatus_conf3,SolveTime_best,Config_best
0,mip1,104.393417,InfeasibleOrUnbounded,152.683291,InfeasibleOrUnbounded,44.068493,Infeasible,78.731996,Infeasible,44.068493,SolveTime_conf2
1,mip2,9600.323414,UserLimit,9600.080699,UserLimit,9600.184597,UserLimit,9600.010288,UserLimit,9600.010288,TIMEOUT
2,mip3,9600.157577,UserLimit,1258.149140,InfeasibleOrUnbounded,3025.400426,Infeasible,2938.074613,Infeasible,1258.149140,SolveTime_conf1
3,mip4,83.344657,InfeasibleOrUnbounded,170.613330,InfeasibleOrUnbounded,54.733004,Infeasible,41.134581,Infeasible,41.134581,SolveTime_conf3
4,mip5,5567.992290,InfeasibleOrUnbounded,220.795008,InfeasibleOrUnbounded,538.053177,Infeasible,300.046997,Infeasible,220.795008,SolveTime_conf1


In [60]:
runtimes = runtimes.drop(columns=['Config_best','SolveTime_best','SolveStatus_default','SolveStatus_conf1','SolveStatus_conf2','SolveStatus_conf3'])
runtimes.head()

,instance_id,SolveTime_default,SolveTime_conf1,SolveTime_conf2,SolveTime_conf3
0,mip1,104.393417,152.683291,44.068493,78.731996
1,mip2,9600.323414,9600.080699,9600.184597,9600.010288
2,mip3,9600.157577,1258.149140,3025.400426,2938.074613
3,mip4,83.344657,170.613330,54.733004,41.134581
4,mip5,5567.992290,220.795008,538.053177,300.046997


In [61]:
runtimes['SolveTime_default'][runtimes['SolveTime_default'] >= 9600] = 9600
runtimes['SolveTime_conf1'][runtimes['SolveTime_conf1'] >= 9600] = 9600
runtimes['SolveTime_conf2'][runtimes['SolveTime_conf2'] >= 9600] = 9600
runtimes['SolveTime_conf3'][runtimes['SolveTime_conf3'] >= 9600] = 9600
runtimes.head()

C:\Users\Jasmin\AppData\Local\Temp\ipykernel_18860\3673047457.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  runtimes['SolveTime_default'][runtimes['SolveTime_default'] >= 9600] = 9600
C:\Users\Jasmin\AppData\Local\Temp\ipykernel_18860\3673047457.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  runtimes['SolveTime_conf1'][runtimes['SolveTime_conf1'] >= 9600] = 9600
C:\Users\Jasmin\AppData\Local\Temp\ipykernel_18860\3673047457.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_gu

,instance_id,SolveTime_default,SolveTime_conf1,SolveTime_conf2,SolveTime_conf3
0,mip1,104.393417,152.683291,44.068493,78.731996
1,mip2,9600.000000,9600.000000,9600.000000,9600.000000
2,mip3,9600.000000,1258.149140,3025.400426,2938.074613
3,mip4,83.344657,170.613330,54.733004,41.134581
4,mip5,5567.992290,220.795008,538.053177,300.046997


In [64]:
runtimes = runtimes[runtimes['instance_id'].isin(id_order)]

In [65]:
runtimes.sort_values(by='instance_id', inplace=True)
runtimes.head()

C:\Users\Jasmin\AppData\Local\Temp\ipykernel_18860\3169124223.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  runtimes.sort_values(by='instance_id', inplace=True)


,instance_id,SolveTime_default,SolveTime_conf1,SolveTime_conf2,SolveTime_conf3
99,mip100,89.865928,779.582903,80.580781,215.399513
999,mip1000,9600.000000,9600.000000,9600.000000,9600.000000
9999,mip10000,22.112561,378.362607,78.919991,18.838446
1000,mip1001,9600.000000,9600.000000,9600.000000,9600.000000
1002,mip1003,963.175574,125.566326,75.395103,101.884567


In [69]:
def make_sorter(l):
    """
    Create a dict from the list to map to 0..len(l)
    Returns a mapper to map a series to this custom sort order
    """
    sort_order = {k:v for k,v in zip(l, range(len(l)))}
    return lambda s: s.map(lambda x: sort_order[x])


runtimes = runtimes.sort_values(by='instance_id', key=make_sorter(id_order))
runtimes

,instance_id,SolveTime_default,SolveTime_conf1,SolveTime_conf2,SolveTime_conf3
2800,mip2801,1500.698079,359.626399,1443.489913,596.350421
4221,mip4222,2555.973815,757.522993,3288.965941,975.140470
2545,mip2546,9600.000000,4165.729887,9600.000000,9600.000000
7003,mip7004,4224.019162,615.969026,885.062022,720.150310
894,mip895,2879.866253,169.917777,244.950671,221.820771
...,...,...,...,...,...
6053,mip6054,9600.000000,9600.000000,9600.000000,9600.000000
2309,mip2310,9600.000000,9600.000000,9600.000000,9600.000000
2535,mip2536,9600.000000,9600.000000,9600.000000,9600.000000
2786,mip2787,9600.000000,9600.000000,9600.000000,9600.000000


In [70]:
new_runtime = runtimes.stack().reset_index()
new_runtime

,level_0,level_1,0
0,2800,instance_id,mip2801
1,2800,SolveTime_default,1500.698079
2,2800,SolveTime_conf1,359.626399
3,2800,SolveTime_conf2,1443.489913
4,2800,SolveTime_conf3,596.350421
...,...,...,...
27220,7246,instance_id,mip7247
27221,7246,SolveTime_default,9600.0
27222,7246,SolveTime_conf1,9600.0
27223,7246,SolveTime_conf2,9600.0


In [71]:
temp = {'instance_id':[], 'repetition':[1]*5445*4, 'algorithm':[],'par10':[],'runstatus':[]}
for index,row in new_runtime.iterrows():
    if row['level_1'] == 'instance_id':
        temp['instance_id'].append(row[0])
        temp['instance_id'].append(row[0])
        temp['instance_id'].append(row[0])
        temp['instance_id'].append(row[0])

    else:
        temp['algorithm'].append(row['level_1'])
        temp['par10'].append(row[0])
        if row[0] == 9600:
            temp['runstatus'].append('timeout')
        else:
            temp['runstatus'].append('ok')

In [72]:
algo_runs_df = pd.DataFrame(temp)
algo_runs_df

,instance_id,repetition,algorithm,par10,runstatus
0,mip2801,1,SolveTime_default,1500.698079,ok
1,mip2801,1,SolveTime_conf1,359.626399,ok
2,mip2801,1,SolveTime_conf2,1443.489913,ok
3,mip2801,1,SolveTime_conf3,596.350421,ok
4,mip4222,1,SolveTime_default,2555.973815,ok
...,...,...,...,...,...
21775,mip2787,1,SolveTime_conf3,9600.000000,timeout
21776,mip7247,1,SolveTime_default,9600.000000,timeout
21777,mip7247,1,SolveTime_conf1,9600.000000,timeout
21778,mip7247,1,SolveTime_conf2,9600.000000,timeout


In [73]:
algo_runs_df.loc[algo_runs_df["algorithm"] == "SolveTime_default", "algorithm"] = 'ConfigDefault'

In [74]:
algo_runs_df.loc[algo_runs_df["algorithm"] == "SolveTime_conf1", "algorithm"] = 'Config1'
algo_runs_df.loc[algo_runs_df["algorithm"] == "SolveTime_conf2", "algorithm"] = 'Config2'
algo_runs_df.loc[algo_runs_df["algorithm"] == "SolveTime_conf3", "algorithm"] = 'Config3'
algo_runs_df.head()

,instance_id,repetition,algorithm,par10,runstatus
0,mip2801,1,ConfigDefault,1500.698079,ok
1,mip2801,1,Config1,359.626399,ok
2,mip2801,1,Config2,1443.489913,ok
3,mip2801,1,Config3,596.350421,ok
4,mip4222,1,ConfigDefault,2555.973815,ok


In [75]:
with open("MIPVerifySDP_cleanARPsamplefeatureset/algorithm_runs.arff","a") as file:
    for index,row in algo_runs_df.iterrows():
        file.write("\n")
        newrow = str(list(row)).strip("[]").replace("'", "")
        file.write(newrow)

    file.close()

### Last one.... make cv.arff

In [76]:
import random

In [77]:
# This script specifies which instances are in which of the 10 folds for 10-fold cross validation
folds_df = pd.DataFrame({'instance_id': id_order, 'repetition':[1]*5445})
folds_df

,instance_id,repetition
0,mip2801,1
1,mip4222,1
2,mip2546,1
3,mip7004,1
4,mip895,1
...,...,...
5440,mip6054,1
5441,mip2310,1
5442,mip2536,1
5443,mip2787,1


In [78]:
# Make a list of length 5445 and randomly replace 0's with numbers from 1 to 10 (each number uses up 10% of the list)
fold_list = []
fold_num = 10

for x in range(1,fold_num+1):
    for _ in range(544):
        fold_list.append(x)


random.seed(1234)
random.shuffle(fold_list)
print(fold_list)
# Check that the lengths are correct
for i in range(1,fold_num+1):
    print(fold_list.count(i))



[3, 8, 2, 6, 3, 1, 8, 2, 4, 5, 1, 7, 6, 9, 9, 4, 2, 8, 4, 1, 4, 5, 6, 6, 10, 8, 9, 10, 5, 10, 6, 1, 1, 1, 10, 3, 7, 2, 3, 7, 3, 6, 6, 2, 5, 10, 9, 2, 10, 4, 9, 1, 6, 7, 8, 3, 7, 5, 10, 2, 4, 8, 9, 6, 2, 8, 2, 2, 6, 9, 10, 4, 9, 1, 10, 7, 4, 6, 7, 2, 5, 9, 10, 2, 3, 4, 5, 2, 6, 3, 4, 3, 9, 5, 8, 9, 3, 3, 8, 6, 6, 10, 5, 3, 8, 2, 9, 4, 2, 2, 2, 1, 7, 4, 4, 6, 2, 3, 4, 4, 2, 8, 4, 1, 4, 6, 4, 8, 6, 3, 5, 6, 8, 3, 3, 6, 1, 6, 9, 8, 1, 5, 3, 6, 4, 5, 10, 3, 10, 2, 2, 3, 3, 8, 1, 3, 8, 5, 7, 3, 3, 2, 8, 8, 6, 7, 9, 8, 1, 3, 9, 10, 2, 5, 2, 8, 7, 9, 8, 10, 5, 1, 7, 3, 7, 9, 9, 3, 2, 4, 10, 4, 8, 9, 5, 3, 7, 7, 3, 5, 4, 8, 4, 3, 6, 3, 5, 4, 2, 9, 10, 5, 8, 8, 6, 5, 3, 10, 5, 10, 7, 9, 8, 5, 7, 3, 7, 3, 7, 2, 5, 1, 1, 6, 3, 5, 3, 2, 6, 9, 4, 9, 9, 4, 3, 9, 2, 1, 5, 9, 6, 4, 2, 9, 4, 5, 10, 9, 10, 1, 3, 2, 1, 1, 5, 6, 6, 8, 9, 9, 5, 9, 8, 7, 6, 10, 1, 1, 6, 1, 2, 5, 2, 9, 10, 4, 2, 3, 4, 1, 4, 5, 4, 6, 10, 1, 10, 1, 3, 8, 4, 8, 5, 1, 9, 10, 6, 5, 10, 2, 2, 5, 2, 1, 2, 3, 3, 4, 10, 7, 4, 7, 6, 9,

In [79]:
fold_list.append(1)
fold_list.append(3)
fold_list.append(5)
fold_list.append(7)
fold_list.append(9)

In [80]:
folds_df['fold'] = fold_list
folds_df.head()

,instance_id,repetition,fold
0,mip2801,1,3
1,mip4222,1,8
2,mip2546,1,2
3,mip7004,1,6
4,mip895,1,3


In [81]:
folds_df

,instance_id,repetition,fold
0,mip2801,1,3
1,mip4222,1,8
2,mip2546,1,2
3,mip7004,1,6
4,mip895,1,3
...,...,...,...
5440,mip6054,1,1
5441,mip2310,1,3
5442,mip2536,1,5
5443,mip2787,1,7


In [82]:
with open("MIPVerifySDP_cleanARPsamplefeatureset/cv.arff","a") as file:
    for index,row in folds_df.iterrows():
        file.write("\n")
        newrow = str(list(row)).strip("[]").replace("'", "")
        file.write(newrow)

    file.close()